In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sb
import os
import glob
import json 
import requests
from bs4 import BeautifulSoup
%matplotlib inline

In [2]:
# os.listdir('./Prediction_train&test_Data')
def load_data(url):
    
    folderName = "./Prediction_train&test_Data/"
    fileName = "Store_league_data.json"
    path = os.path.join(folderName, fileName)

#     if os.path.exists(path):
#         files = glob.glob("./Prediction_train&test_Data/*")
#         for f in files:
#             os.remove(f)

    response = requests.get(url)
    # print(path)

    with open(path, 'wb') as outfile:
        outfile.write(response.content)
     

        
url = "https://vsmobile.bet9ja.com/shopadmin/standings_view/league_data.php?pid=14001"
load_data(url)

In [3]:
folderName = "./Prediction_train&test_Data/"
fileName = "Store_league_data.json"


path = os.path.join(folderName, fileName)


with open(path, "r") as outfile:
    json_obj = json.load(outfile)

In [4]:
# print(json_obj)

In [5]:
# json_obj["teams"]

In [6]:
teams = {"id": [], "name": [], "club": []}

In [7]:
for x, y in json_obj["teams"].items():
    # main_info.id.append(x)
    teams["id"].append(x)
    teams["name"].append(y["fifa"])
    teams["club"].append(y["team"])
   
    
    # print(x)
    # print(y)

In [8]:
# teams # if you have ound its index you havefound the club

In [9]:
# len(teams['id']) # total number of teams 

In [10]:
# len(json_obj["weeks"])

In [11]:
# print(json_obj["weeks"]["1"]["event_data"]), print(len(json_obj["weeks"]["1"]["event_data"]))

In [12]:
# print(json_obj["weeks"]["1"]["teamstats"].split("-"))

In [13]:
def indetify_club(teams_id):
    idx = teams["id"].index(teams_id)
    return teams["name"][idx]

def get_club_id(team_name):
    club_id = teams["name"].index(team_name)
    return teams["id"][club_id]

In [14]:
# indetify_club("50")

In [15]:
# json_obj["weeks"]["1"]

In [16]:
def query_score(teamA, teamB):
    if teamA > teamB:
        return "win-loss"
    elif teamA < teamB:
        return "loss-win"
    else:
        return "draw-draw"

In [17]:
def get_new_data(data, score, week):
    start = 0
    stop = 1
    add_num = 2
    data = data.split("-")
    score = score.split("-")
    
    # store clubs id and name.
    result = {
        "weeks": [],
        "id":[],
        "club": [],
        "score_string": [],
        "score_num": [],
        "game_type": []
    }
    
    for _ in range(10):
        # slice the event_data from data pass thoure the function parameter
        teamA = data[start]
        teamB = data[stop]

        # identfy the club from thire respected ID
        clubA = indetify_club(teamA)
        clubB = indetify_club(teamB)
		
        # slice the teamstats from score pass thoure the function parameter
        teamA_score = score[start]
        teamB_score = score[stop] 
        
        
        result["weeks"].append(week)
        result["weeks"].append(week)
        
		# appending the data twice to add information
        
        ####### HOME APPENDING #######
        result["id"].append(f"{teamA}")
        
        result["club"].append(f"{clubA}")
        # query scores
        win_loss_drawA = query_score( int(teamA_score), int(teamB_score) ).split("-")[0]
        
        result["score_string"].append( win_loss_drawA )
        # query wins scores
        result["score_num"].append( f" {int(teamA_score)} " )
        
        result["game_type"].append( "home" )
        
        
        
        ####### AWAY APPENDING #######
        result["id"].append(f"{teamB}")
        
        result["club"].append(f"{clubB}")
        
        win_loss_drawB = query_score( int(teamA_score), int(teamB_score) ).split("-")[1]
        
        result["score_string"].append( win_loss_drawB )
        
        result["score_num"].append( f" {int(teamB_score)} " )
        
        result["game_type"].append( "away" )
        
        
        # accumilate the START and STOP for the next sclice
        start += add_num
        stop += add_num

        
        
    return result
 

In [18]:
# get_new_data( json_obj["weeks"]["1"]["event_data"], json_obj["weeks"]["1"]["teamstats"],  json_obj["weeks"]["1"]["matchday"])

In [19]:
main_second_info = []
for week, events in json_obj["weeks"].items():
    main_second_info.append(get_new_data( events["event_data"], events["teamstats"], events["matchday"]) )
    
   # print(week)

In [20]:
# json_obj["weeks"]

In [21]:
# main_second_info[0]

# create database

In [22]:
df_list = []
for data in main_second_info:
    new_data = pd.DataFrame(data)
    df_list.append(new_data)

In [23]:
df = pd.concat(df_list)

In [24]:
# df.head()

### Wrangling Data

In [25]:
df["binary_game_type"] = df["game_type"].apply(lambda x: 0 if x == "home" else 1)

In [26]:
df.head()

,weeks,id,club,score_string,score_num,game_type,binary_game_type
0,1,45,LIV,win,3,home,0
1,1,827,LEE,loss,0,away,1
2,1,53,NWC,win,3,home,0
3,1,60,ASV,loss,0,away,1
4,1,51,TOT,win,3,home,0


In [27]:
# df["weeks"].apply(lambda x: int(x))

### ANALYING DATA

In [28]:
# color_paletter = sb.color_palette()[0]
# sb.barplot(x = df["score_string"], y = df["weeks"].apply(lambda x: int(x)) , color=color_paletter)

In [29]:
# df.groupby("club").mean()
# df.head()

In [30]:
# sb.barplot(x = df["club"], y = df["score_string"] , color=color_paletter)
all_tems = df[["club", "score_string"]].value_counts()
# df.query("club == 'MNU'")
club = all_tems.index
club_values = all_tems.values
# plt.figure(figsize=[10, 6])
main_club = [x[0] for x in club]
main_scores = [x[1] for x in club]
print(f"main_club: {main_club[0]}, --- main_scores: {main_scores[0]}, ---, club_values: {club_values[0]}")

main_club: BOU, --- main_scores: loss, ---, club_values: 35


In [31]:
query_df_club = pd.DataFrame({
    "clubs": main_club,
    "scores": main_scores,
    "occ": club_values
})

In [32]:
# query_df_club

In [33]:
# plt.figure(figsize=[20, 10])
# sb.barplot(data=query_df_club, x="clubs", y="occ", hue="scores")

In [34]:
query_df_club_wins = query_df_club.query("scores == 'win'")
query_df_club_loss = query_df_club.query("scores == 'loss'")
query_df_club_draw = query_df_club.query("scores == 'draw'")
# query_df_club_draw.head()

In [35]:
# plt.figure(figsize=[20, 10])
# plt.title(" WINS ")
# sb.barplot(data=query_df_club_wins, x="clubs", y="occ", color=color_paletter)

In [36]:
# plt.figure(figsize=[20, 10])
# plt.title(" LOSS ")
# sb.barplot(data=query_df_club_loss, x="clubs", y="occ", color=color_paletter)

In [37]:
# df.head()

In [38]:
# plt.figure(figsize=[20, 10])
# plt.title(" DRAWS ")
# sb.barplot(data=query_df_club_draw, x="clubs", y="occ", color=color_paletter)

In [39]:
# sort_count = df.score_string.value_counts()

# plt.figure(figsize=[20, 10])

# plt.pie([np.float64(x) for x in sort_count.values], labels=sort_count.index, startangle=90, counterclock=False);


### Select training data

In [40]:
# df.head()

In [41]:
train_df = df[["weeks", "id", "score_string", "binary_game_type"]]
# train_df = df[["club", "final"]]

In [42]:
train_df = train_df.reset_index()
train_df = train_df.drop('index', axis = 1) 
# train_df.head()

In [43]:
# len(train_df)

#### Train the model

In [44]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

#### SPLIT DATA FOR EVEN FEATURES

In [45]:
# train_df[["home", "away"]][:20]["home"].value_counts()
# prec_20 = int((20/100)*len(train_df))
# prec_20

In [46]:
# train_df[["home", "away"]][:20]["away"].value_counts()

In [47]:
# train_data = train_df[prec_20:]
# test_data = train_df[:prec_20]

In [48]:
# train_data.head()

### Ordinary Split

In [49]:
# x_train_data = train_data[["id"]]
# x_train_data_dummy = pd.get_dummies(x_train_data)
# y_train_data = train_data["score_string"]

In [50]:
# x_test_data = test_data[["id"]]
# x_test_data_dummy = pd.get_dummies(x_test_data)
# y_test_data = test_data["score_string"]

### cross validation spilt

In [51]:
# x_train, x_test, y_train, y_test = train_test_split(train_df[["id"]], train_df["score_string"], test_size=0.3)
# x_train, x_test, y_train, y_test = train_test_split(train_df["club"], train_df["final"], test_size=0.3)

### Stratified Shuffle Split

In [52]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
for train_index, test_index in split.split(train_df[["weeks", "id", "binary_game_type"]], train_df["score_string"]):
    strat_train_set = train_df.loc[train_index]
    strat_test_set = train_df.loc[test_index]    

In [53]:
# y_train.head()

In [54]:
# x_train.head()

In [55]:
# trat_train_set.head()

In [56]:
# strat_test_set.head()

### StratifiedShuffleSplit train

In [57]:
y_strat_train_set = strat_train_set["score_string"]
x_strat_train_set = strat_train_set[["weeks", "id", "binary_game_type"]]
# x_strat_train_set = strat_train_set["club"]

In [58]:
# len(y_strat_train_set), len(x_strat_train_set)

### StratifiedShuffleSplit test

In [59]:
y_strat_test_set = strat_test_set["score_string"]
x_strat_test_set = strat_test_set[["weeks", "id", "binary_game_type"]]
# x_strat_test_set = strat_test_set["club"]

In [60]:
# len(y_strat_train_set), len(x_strat_train_set)

In [61]:
# x_strat_train_set, x_strat_test_set, y_strat_train_set, y_strat_test_set

### Selecting and Training model

##### Decision tree classifer

In [62]:
model = DecisionTreeClassifier()
model.fit(x_strat_train_set, y_strat_train_set)
# x_strat_train_set, x_strat_test_set, y_strat_train_set, y_strat_test_set

DecisionTreeClassifier()

In [63]:
predictions = model.predict(x_strat_test_set)


In [64]:
# len(predictions) #, len(y_strat_test_set), len(x_strat_test_dummy)
# y_strat_test_set

In [65]:
accuracy = accuracy_score(y_strat_test_set, predictions)

In [66]:
print(f"accuracy: {accuracy}")

accuracy: 0.581081081081081


##### SGDClassifier

In [67]:
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(x_strat_train_set, y_strat_train_set)
# x_strat_train_set, x_strat_test_set, y_strat_train_set, y_strat_test_set

SGDClassifier(random_state=42)

In [68]:
sgd_predict = sgd_clf.predict(x_strat_test_set)
sgd_accuracy = accuracy_score(y_strat_test_set, sgd_predict)

In [69]:
print(f"accuracy: {sgd_accuracy}")

accuracy: 0.47297297297297297


##### RandomForestClassifier & cross_validation

In [70]:
forest_clf = RandomForestClassifier(random_state=42)

In [71]:
from sklearn.model_selection import cross_val_score
main_data = cross_val_score(sgd_clf, x_strat_train_set, y_strat_train_set, cv=7, scoring="accuracy")
print(main_data)
# x_strat_train_set, x_strat_test_set, y_strat_train_set, y_strat_test_set

[0.47058824 0.47058824 0.58823529 0.55294118 0.04761905 0.64285714
 0.5952381 ]


In [72]:
main_data_forst_clf = cross_val_score(forest_clf, x_strat_train_set, y_strat_train_set, cv=7, scoring="accuracy")
# x_train_data_dummy, y_train_data, x_test_data_dummy, y_test_data
main_data_forst_clf

array([0.61176471, 0.61176471, 0.58823529, 0.48235294, 0.61904762,
       0.69047619, 0.64285714])

In [73]:
main_data.mean(), main_data_forst_clf.mean()

(0.4811524609843937, 0.6066426570628253)

#### Gride Search Cross Validation

In [74]:
from sklearn.model_selection import GridSearchCV

#### Random Forest Gride Search Cross Validation 

In [75]:
param_grid = [
    {"n_estimators": [3, 10, 30], 
     # "max_features": [0, 0, 0]
    },
    {"bootstrap": [False], "n_estimators": [3, 10], 
     # "max_features": [2, 3, 4]
    }
]
grid_search = GridSearchCV(forest_clf, param_grid, cv=7, scoring="accuracy")

In [76]:
grid_search.fit(x_strat_train_set, y_strat_train_set)
# x_strat_train_set, x_strat_test_set, y_strat_train_set, y_strat_test_set

GridSearchCV(cv=7, estimator=RandomForestClassifier(random_state=42),
             param_grid=[{'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'n_estimators': [3, 10]}],
             scoring='accuracy')

In [77]:
grid_search.best_params_

{'n_estimators': 30}

In [78]:
final_model = grid_search.best_estimator_
final_model

RandomForestClassifier(n_estimators=30, random_state=42)

In [79]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(mean_score), params)

0.7377111135633174 {'n_estimators': 3}
0.763644721912305 {'n_estimators': 10}
0.7646396991706742 {'n_estimators': 30}
0.7433444856927129 {'bootstrap': False, 'n_estimators': 3}
0.7535164993853409 {'bootstrap': False, 'n_estimators': 10}


#### Support Vectore Maching Gride Search Cross Validation 

In [80]:
svm_clf = SVC()
svm_parames = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
svm_gridCV = GridSearchCV(SVC(), svm_parames, refit = True, cv=7, scoring="accuracy")

svm_gridCV.fit(x_strat_train_set, y_strat_train_set)

GridSearchCV(cv=7, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             scoring='accuracy')

In [81]:
svm_gridCV.best_params_

{'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}

In [82]:
final_model_svc = svm_gridCV.best_estimator_
final_model_svc

SVC(C=1, gamma=0.01)

In [83]:
svm_cvres = svm_gridCV.cv_results_
for mean_score, params in zip(svm_cvres["mean_test_score"], svm_cvres["params"]):
    print(np.sqrt(mean_score), params)

0.7376297438794224 {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
0.7602836394958095 {'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}
0.7991592220483187 {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
0.7809506729502399 {'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}
0.7809506729502399 {'C': 0.1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.7700241391475646 {'C': 1, 'gamma': 1, 'kernel': 'rbf'}
0.7959983953364174 {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
0.8044245393670025 {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
0.7938463811284475 {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.7809506729502399 {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.773329952663334 {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
0.7432502730489609 {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
0.8034041220166283 {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
0.795960690843805 {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.7809506729502399 {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.773329952663334 {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
0.7248518221441029 {'C': 100, 'g

### FINAL PREDICTION!!!🎉👏🏽🎊

In [84]:
# x_train_data_dummy, y_train_data, x_test_data_dummy, y_test_data
# x_strat_train_set, x_strat_test_set, y_strat_train_set, y_strat_test_set

final_predctions = final_model.predict(x_strat_test_set)
final_accuracy_score = accuracy_score(y_strat_test_set, final_predctions)
print(f"accuracy: {final_accuracy_score}")

accuracy: 0.5743243243243243


In [98]:
svc_final_predctions = final_model_svc.predict(x_strat_test_set)
svc_final_accuracy_score = accuracy_score(y_strat_test_set, svc_final_predctions)
print(f"accuracy: {svc_final_accuracy_score}")

accuracy: 0.6283783783783784


#### Main predicions!! 

In [85]:
id_cubs = np.array([get_club_id(x) for x in df.club.value_counts().index]).reshape(-1, 1)
cubs = df.club.value_counts().index

##### SELECTING MAIN TEAMS FOR PREDICTIONS

In [86]:
df.club.value_counts().index

Index(['MNC', 'SOU', 'LEI', 'BOU', 'TOT', 'FUL', 'CHE', 'FOR', 'WHU', 'LEE',
       'ARS', 'EVE', 'WOL', 'NWC', 'BRN', 'ASV', 'MNU', 'LIV', 'BRI', 'CRY'],
      dtype='object')

In [87]:
main_clubs = ["BRI", "LEE", "WOL", "BRN"]
teams_id = [get_club_id(x) for x in main_clubs]
game_type = ["home", "away", "home", "away"]
weaks = [len(df)/20 for _ in range(4)]

In [88]:
x_strat_train_set.head()

,weeks,id,binary_game_type
266,14,50,0
405,21,827,1
725,37,512,1
617,31,63,1
516,26,49,0


##### Combine Data For DataFrame

In [89]:
prediction_data = pd.DataFrame({
    "weeks": weaks,
    "id": teams_id,
    "binary_game_type": [ 0 if x == "home" else 1 for x in game_type ]
})
prediction_data

,weeks,id,binary_game_type
0,37.0,71,0
1,37.0,827,1
2,37.0,525,0
3,37.0,972,1


In [90]:
rfc_predictions = final_model.predict(prediction_data)
svm_predictio

In [91]:
# [get_club_id(x) for x in df.club.value_counts().index]

In [92]:
# plt.figure(figsize=[20, 10])
# sb.barplot(data=query_df_club, x="clubs", y="occ", hue="scores")
# plt.xlabel(fontsize=25)
# plt.xlabel('xlabel', fontsize=18);

In [93]:
prediction_df = pd.DataFrame({
    "week": weaks,
    "id_clubs": teams_id,
    "clubs": main_clubs,
    "rf_predictions": rfc_predictions,
    "svm_predictios": 
    "game_type": game_type
})
prediction_df
# .sort_values(by=["clubs"])

,week,id_clubs,clubs,predictions,game_type
0,37.0,71,BRI,loss,home
1,37.0,827,LEE,loss,away
2,37.0,525,WOL,loss,home
3,37.0,972,BRN,loss,away


In [94]:
print(f"week of game {len(df)/20}")

week of game 37.0


In [95]:
# indetify_club
# new_df = pd.DataFrame({"home": [indetify_club("MNC"), indetify_club("LEI"), indetify_club("LIV"), indetify_club("NWC")], 
  #                     "away": [indetify_club("BRI"), indetify_club("CHE"), indetify_club("TOT"), indetify_club("FOR")]})

### PRE PER PREDICTIONS.

In [96]:
# d4 = BeautifulSoup
url = "https://bam.nr-data.net/jserrors/1/c95cd51526?a=214311961&v=1223.PROD&to=ZlBXZxcAVkEHV0NbDV8aYEEMTlpXEg1dU09cWldaCQQXXglTXlxNWFtRVh1PSFoW&rst=476457&ck=0&s=0&ref=https://vsmobile.bet9ja.com/bet9ja-mobile/login/"
html = requests.get(url)
print(html.text)

s = BeautifulSoup(html.content, "html.parser")
# requests

GIF89a     ,       


In [97]:
url = https://vsmobile.bet9ja.com" + uri + (containsChar('?', uri) ? "&mode=" : "?mode=") + mode + "&lang=" + lang;
    
url = "https://vsmobile.bet9ja.com/mobile/themes/?sk=bet9ja&t=b61c29e6-9348-4c58-af90-378760a74693&game=league_premier&pid=14001,14003,14011,14012,14014,14015,14016,14017&v=0&mode=Premier&lang=en_GB";
url = "https://vsmobile.bet9ja.com/shopadmin/standings_view/league_data.php?pid=14003"

SyntaxError: invalid syntax (<ipython-input-97-5e331335d67e>, line 1)

In [ ]:
new_df = pd.DataFrame({"home": ["for"], "away": ["che"]})
new_df

In [ ]:
pd.get_dummies(new_df)

In [ ]:
# sgd_clf.predict(pd.get_dummies(new_df))